In [32]:
import scipy
import numpy as np
import pandas

Исследуется заработная плата работников с одинаковой квалификацией на предприятиях одной отрасли. С этой целью были проведены четыре выборки среди предприятий отрасли. Результаты представлены в таблицах  19 –22. Для предприятий этой отрасли была установлена средняя заработная плата 22 500 руб. 

In [33]:
# Таблица 20
x20 = np.array([14600, 22700, 23600, 24600, 24700, 25400, 25550, 26550, 27400])
n20 = np.array([1, 4, 5, 6, 7, 2, 4, 1, 3])

По результатам  проведённого статистического анализа ответьте на вопросы:

* Можно ли считать, что заработная плата на предприятии соответствует установленному уровню?
* Сравните полученные выборочные с данными другого предприятия отрасли (по выбору).  Можно ли сказать, что работники этих предприятий получают одинаковое вознаграждение?
* Насколько нужно увеличить объём  выборки, чтобы точность оценки увеличились на 15%?

In [34]:
# Среднее арифметическое 
mean20 = (x20 * n20).sum() / sum(n20)
mean20

24413.636363636364

По ЦПТ при существовании дисперсии распределения оценка

$$\sqrt{n} \frac{\bar{X}_{n}-\mu}{\sigma} \rightarrow N(0,1), \text { по распределению при } n \rightarrow \infty,
$$

где $\bar{X}_n$ среднее арифметическое, $\mu$ - мат ожидание, $\sigma^2$ - дисперсия. Мы можем заменить $\sigma$ на стандартное отклонение, так как стандартное отклонение есть состоятельная оценка $\sigma$ и асимптотическая нормальность не изменится. Отсюда мы можем получить асимптотический доверительный интервал уровня $\gamma$

$$\bar{X}-\frac{z_{\frac{1+\gamma}{2}} \bar s}{\sqrt{n}}<a<\bar{X}+ \frac{z_{\frac{1+\gamma}{2}} \bar s}{\sqrt{n}}$$

In [35]:
from scipy.stats import norm

def confidence_interval(data, gamma):
    mean = np.mean(data)
    std = np.std(data)
    border = norm.ppf((1 + gamma) / 2) * std / np.sqrt(len(data))
    return mean - border, mean + border

In [36]:
# Приведем данные в удобный для python вид
data20 = []
for salary, num_employ in zip(x20, n20):
    data20 += [salary] * num_employ

data20 = np.array(data20)
print(data20)

[14600 22700 22700 22700 22700 23600 23600 23600 23600 23600 24600 24600
 24600 24600 24600 24600 24700 24700 24700 24700 24700 24700 24700 25400
 25400 25550 25550 25550 25550 26550 27400 27400 27400]


Мы видим, что ожидаемая средняя зараплата не лежит в 99-ти процентном доверительном интервале, то есть заработная плата на предприятии не соответствует установленному уровню.

In [37]:
confidence_interval(data20, 0.99)

(23455.8215664271, 25371.451160845627)

Критерий Вилкоксона для равенства медианы data22  - 22 500

In [38]:
from scipy.stats import wilcoxon

# H0: med X = mo
wilcoxon(data20 - 22_500)

WilcoxonResult(statistic=33.0, pvalue=9.23558212695375e-06)

Отвергаем нулевую гипотезу при уровне значимости 0.005. Среднее в смысле медианы не равно 22 500

Для оценивания средних зарплат 2ух предприятий будем использовать критерий равенства медиан, потому что она лучше оценивает среднее в случае зарплат (не сильно реагирует на выбросы). В теории можно было сделать доверительный интервал используя медиану, но мы не знаем асимптотическое распределение статистики.

In [39]:
x21 = np.array([21750, 21800, 22100, 22200, 22900, 23100, 24550, 25500, 25600])
n21 = np.array([1, 3, 3, 5, 6, 3, 2, 1, 1])

data21 = []
for salary, num_employ in zip(x21, n21):
    data21 += [salary] * num_employ

data21 = np.array(data21)
print(data21)

[21750 21800 21800 21800 22100 22100 22100 22200 22200 22200 22200 22200
 22900 22900 22900 22900 22900 22900 23100 23100 23100 24550 24550 25500
 25600]


In [41]:
# Доверительный интервал для средних зарплат 21 цеха. 
# Он не пересекается с доверительным интервалом 20ого цеха

confidence_interval(data21, 0.99)

(22300.254449246793, 23407.745550753207)

In [45]:
from tqdm.auto import tqdm

In [52]:
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.median, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, # чтобы сохранить дисперсию, берем такой же размер выборки
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) 
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

In [56]:
get_bootstrap(pd.DataFrame(data20), pd.DataFrame(data21), bootstrap_conf_level=0.99)['p_value']

1.5213663255675115e-07

Отвергаем нулевую гипотезу при уровне значимости 0.005. Средние в смысле медианы не равны.